In [16]:
import pandas as pd
import json
import plotly.express as px

In [17]:
with open('MANY_OUTLIERS.json') as file:
    data = json.load(file)


In [18]:
rbf_pts = pd.DataFrame(data['rbfpoints'], columns=['i', 'j', 'x', 'y'])
rbf_pts['name'] = 'rbfpoints'

In [19]:
fig = px.scatter(rbf_pts, x="x", y="y", hover_data={
    "i":':.0f', 
    "j":':.0f',
    "x":':.1f',
    "y":':.1f',
    }, color="name")

fig.update_layout(
    yaxis = dict(autorange="reversed")
)

fig.show()


In [20]:
rbf_pts

,i,j,x,y,name
0,-7,-3,311.564232,1440.797321,rbfpoints
1,-8,-4,231.325562,1359.975453,rbfpoints
2,6,-10,1353.785066,881.681599,rbfpoints
3,6,9,1353.779820,2401.682088,rbfpoints
4,-7,2,311.564101,1840.797293,rbfpoints
...,...,...,...,...,...
575,-7,12,311.569440,2640.808073,rbfpoints
576,10,-6,1649.280106,1197.479679,rbfpoints
577,-9,-5,151.058115,1278.616681,rbfpoints
578,-9,7,151.058155,2238.613709,rbfpoints


In [21]:
import pandas as pd
import numpy as np

# Supponiamo che rbf_pts sia il DataFrame di input
# Aggiungiamo una colonna di indici espliciti
rbf_pts['index'] = rbf_pts.index

# Creiamo delle copie delle colonne 'i' e 'j' per facilitare le operazioni di merge successivo
rbf_pts['key_i'] = rbf_pts['i']
rbf_pts['key_j'] = rbf_pts['j']

# Adiacenti j (up e down)
j_up = rbf_pts.copy()
j_up['j'] = j_up['j'] - 1  # punti adiacenti in j verso l'alto
merged_up = rbf_pts.merge(j_up, on=['i', 'j'], suffixes=('', '_up'))

j_down = rbf_pts.copy()
j_down['j'] = j_down['j'] + 1  # punti adiacenti in j verso il basso
merged_down = rbf_pts.merge(j_down, on=['i', 'j'], suffixes=('', '_down'))

# Adiacenti i (left e right)
i_left = rbf_pts.copy()
i_left['i'] = i_left['i'] - 1  # punti adiacenti in i verso sinistra
merged_left = rbf_pts.merge(i_left, on=['i', 'j'], suffixes=('', '_left'))

i_right = rbf_pts.copy()
i_right['i'] = i_right['i'] + 1  # punti adiacenti in i verso destra
merged_right = rbf_pts.merge(i_right, on=['i', 'j'], suffixes=('', '_right'))

# Aggiungere le liste degli indici degli adiacenti
rbf_pts['adiacenti_j'] = rbf_pts.apply(lambda row: merged_up.loc[merged_up['index'] == row['index'], 'index_up'].tolist(
) + merged_down.loc[merged_down['index'] == row['index'], 'index_down'].tolist(), axis=1)
rbf_pts['adiacenti_i'] = rbf_pts.apply(lambda row: merged_left.loc[merged_left['index'] == row['index'], 'index_left'].tolist(
) + merged_right.loc[merged_right['index'] == row['index'], 'index_right'].tolist(), axis=1)

# Ora rimuoviamo le colonne temporanee
rbf_pts.drop(columns=['key_i', 'key_j'], inplace=True)

print(rbf_pts)

      i   j            x            y       name  index  j_dist_up  \
0    -7  -3   311.564232  1440.797321  rbfpoints      0  80.000622   
1    -8  -4   231.325562  1359.975453  rbfpoints      1  79.999400   
2     6 -10  1353.785066   881.681599  rbfpoints      2  79.999972   
3     6   9  1353.779820  2401.682088  rbfpoints      3  79.997887   
4    -7   2   311.564101  1840.797293  rbfpoints      4  80.000858   
..   ..  ..          ...          ...        ...    ...        ...   
575  -7  12   311.569440  2640.808073  rbfpoints    575        NaN   
576  10  -6  1649.280106  1197.479679  rbfpoints    576        NaN   
577  -9  -5   151.058115  1278.616681  rbfpoints    577        NaN   
578  -9   7   151.058155  2238.613709  rbfpoints    578        NaN   
579  10   2  1649.276735  1837.480370  rbfpoints    579        NaN   

     j_dist_down  i_dist_left  i_dist_right adiacenti_j adiacenti_i  
0     -79.999587    80.219522    -80.238761  [206, 279]  [332, 281]  
1     -79.999360   

In [22]:
rbf_pts

,i,j,x,y,name,index,j_dist_up,j_dist_down,i_dist_left,i_dist_right,adiacenti_j,adiacenti_i
0,-7,-3,311.564232,1440.797321,rbfpoints,0,80.000622,-79.999587,80.219522,-80.238761,"[206, 279]","[332, 281]"
1,-8,-4,231.325562,1359.975453,rbfpoints,1,79.999400,-79.999360,80.238464,-80.267210,"[281, 207]","[279, 530]"
2,6,-10,1353.785066,881.681599,rbfpoints,2,79.999972,-79.999683,79.741195,-80.151200,"[225, 430]","[208, 480]"
3,6,9,1353.779820,2401.682088,rbfpoints,3,79.997887,-80.000481,79.754863,-80.143051,"[377, 265]","[454, 333]"
4,-7,2,311.564101,1840.797293,rbfpoints,4,80.000858,-80.000345,80.219922,-80.238559,"[42, 112]","[151, 41]"
...,...,...,...,...,...,...,...,...,...,...,...,...
575,-7,12,311.569440,2640.808073,rbfpoints,575,NaN,NaN,NaN,NaN,[340],"[487, 526]"
576,10,-6,1649.280106,1197.479679,rbfpoints,576,NaN,NaN,NaN,NaN,"[510, 569]",[446]
577,-9,-5,151.058115,1278.616681,rbfpoints,577,NaN,NaN,NaN,NaN,"[530, 512]",[207]
578,-9,7,151.058155,2238.613709,rbfpoints,578,NaN,NaN,NaN,NaN,"[531, 513]",[473]


In [27]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Supponiamo che rbf_pts sia il DataFrame con le colonne già calcolate (i_dist, j_dist, adiacenti_i, adiacenti_j)
plot_data = []

# Iteriamo sui punti e aggiungiamo i loro adiacenti
for idx, row in rbf_pts.iterrows():
    # Aggiungi il punto principale
    plot_data.append({'x': row['x'], 'y': row['y'], 'i': row['i'], 'j': row['j'], 'group': f'Point {idx}'})
    
    # Aggiungi i punti adiacenti
    adiacenti_i = row['adiacenti_i']
    adiacenti_j = row['adiacenti_j']
    
    for adj_idx in adiacenti_i + adiacenti_j:
        adj_point = rbf_pts.loc[adj_idx]
        plot_data.append({'x': adj_point['x'], 'y': adj_point['y'], 'i': adj_point['i'], 'j': adj_point['j'], 'group': f'Point {idx}'})

# Convertiamo la lista in un DataFrame
plot_data_df = pd.DataFrame(plot_data)

# Creare il grafico in coordinate x, y
fig_xy = go.Figure()

# Aggiungi tutti i punti con colore nero come prima entry della legenda
fig_xy.add_trace(go.Scatter(x=rbf_pts['x'], y=rbf_pts['y'], mode='markers', 
                            marker=dict(color='black', size=5), name='All Points'))

# Aggiungi ogni gruppo di punti con i loro adiacenti
for group in plot_data_df['group'].unique():
    group_data = plot_data_df[plot_data_df['group'] == group]
    fig_xy.add_trace(go.Scatter(x=group_data['x'], y=group_data['y'], mode='markers',
                                marker=dict(size=10), name=group))

# Configurazione del layout
fig_xy.update_layout(title="Punti e i loro adiacenti (coordinate x, y)", xaxis_title="x", yaxis_title="y")

# Creare il grafico in coordinate i, j
fig_ij = go.Figure()

# Aggiungi tutti i punti con colore nero come prima entry della legenda
fig_ij.add_trace(go.Scatter(x=rbf_pts['i'], y=rbf_pts['j'], mode='markers', 
                            marker=dict(color='black', size=5), name='All Points'))

# Aggiungi ogni gruppo di punti con i loro adiacenti
for group in plot_data_df['group'].unique():
    group_data = plot_data_df[plot_data_df['group'] == group]
    fig_ij.add_trace(go.Scatter(x=group_data['i'], y=group_data['j'], mode='markers',
                                marker=dict(size=10), name=group))

# Configurazione del layout
fig_ij.update_layout(title="Punti e i loro adiacenti (coordinate i, j)", xaxis_title="i", yaxis_title="j")

# Mostra entrambi i grafici
fig_xy.show()
fig_ij.show()
